In [2]:
import os
# Force unsloth to use the local GPU memory efficiently
# os.environ["UNSLOTH_RETURN_LOGITS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

In [1]:
# run `setup_grpo_transformers.sh` first

# OR Install required packages (needed after Space restarts)
%pip install -q huggingface_hub transformers trl peft accelerate bitsandbytes datasets tensorboard dotenv
%pip install httpx==0.27.2

Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 76 kB 10.0 MB/s eta 0:00:01
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
Note: you may need to restart the kernel to use updated packages.


### Local login, not for use with spaces

### Server-Side HF Login

In [2]:
# Set remote HF_TOKEN from local .env
import os
from dotenv import load_dotenv

load_dotenv()
hf_token = os.getenv('HF_TOKEN')

# ssh -i ~/.ssh/id_ed25519 dataimaginations-heirarchical-reasoning@ssh.hf.space "echo 'export HF_TOKEN={hf_token}' >> ~/.bashrc"
print("✅ Token set! Restart remote shell to activate.")

✅ Token set! Restart remote shell to activate.


In [3]:
import os
from huggingface_hub import login

# Login using your HF token
hf_token = os.getenv('HF_TOKEN')  # Try environment variable first

if hf_token:
    # login(token=hf_token)
    login()
    print("✅ Logged in with HF_TOKEN environment variable")
else:
    # If no env var, prompt for token (you'll need to paste it)
    login()
    print("✅ Logged in interactively")

✅ Logged in with HF_TOKEN environment variable


In [ ]:
# !ssh -i ~/.ssh/id_ed25519 dataimaginations-heirarchical-reasoning@ssh.hf.space "echo 'export HF_TOKEN=' >> ~/.bashrc"

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
from trl import GRPOConfig, GRPOTrainer

# --- CONFIGURATION ---
MODEL_NAME = "google/gemma-2-2b-it"
output_dir = "gemma-2-2b-reasoning-v1"

print("⏳ Loading model in 4-bit...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    # No attn_implementation - let it auto-select
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Fix padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.pad_token_id

print(f"✅ Loaded {MODEL_NAME}")# Changed from flash_attention_2


/tmp/ipykernel_732/3638208885.py:4: FutureWarning: Support for Python 3.9 will be dropped in the next release (after its end-of-life on October 31, 2025). Please upgrade to Python 3.10 or newer.
  from trl import GRPOConfig, GRPOTrainer


⏳ Loading model in 4-bit...


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

✅ Loaded google/gemma-2-2b-it


In [6]:
# 2. Prepare Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.padding_side = "left" # CRITICAL for reasoning/generation steps!
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [7]:
# Fix tokenizer padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.pad_token_id

print(f"✅ Pad token set to: {tokenizer.pad_token}")
print(f"✅ Vocab size: {len(tokenizer)}")

# Gemma3 has a nested config structure (vision + text)
if hasattr(model.config, 'text_config'):
    print(f"✅ Model vocab size: {model.config.text_config.vocab_size}")
else:
    print(f"✅ Model vocab size: {model.config.vocab_size}")

✅ Pad token set to: <pad>
✅ Vocab size: 256000
✅ Model vocab size: 256000


In [8]:
# Cell 3: The HICRA Logic (Strategic Grams)

# These are the "thinking words" the paper identified. 
# When the model uses these, it is "planning".
STRATEGIC_GRAMS = [
    "first i need to", "let's look at", "alternatively", "wait", 
    "but i'm not sure", "let's see if", "notice that", 
    "the final answer is", "let's assume", "we can conclude",
    "implies that", "to solve this", "break it down", 
    "suppose that", "checking the", "recall that"
]

def correctness_reward_func(prompts, completions, answer, **kwargs):
    """
    Give reward=1.0 if completion matches answer, 0.0 otherwise
    """
    rewards = []
    for completion in completions:
        # Extract just the number from the completion
        # (model might generate extra text like "The answer is 14")
        reward = 1.0 if str(answer).strip() in completion.strip() else 0.0
        rewards.append(reward)
    return rewards

def hicra_planning_reward_func(prompts, completions, **kwargs):
    """
    HICRA Proxy: Gives a small bonus for using 'Strategic Grams'.
    This encourages the model to 'think' before answering.
    """
    rewards = []
    for completion in completions:
        score = 0.0
        # Check for presence of planning words
        completion_lower = completion.lower()
        for gram in STRATEGIC_GRAMS:
            if gram in completion_lower:
                score += 0.1 # Small bonus for EACH planning step
        
        # Cap the bonus so it doesn't game the system just by spamming words
        rewards.append(min(score, 0.5)) 
    return rewards

In [10]:
# 1. Load BOTH Datasets
from datasets import load_dataset

dataset_train = load_dataset("json", data_files="reasoning_dataset_v2_train.json", split="train", download_mode="force_redownload")
dataset_test = load_dataset("json", data_files="reasoning_dataset_v2_test.json", split="train", download_mode="force_redownload") # Load as 'train' split then rename if needed, but 'train' split works fine for just passing data
# For later: nvidia/Nemotron-Post-Training-Dataset-v1

# GRPO expects a specific format. We don't need a system prompt for simple math.
# It just needs 'prompt' and 'answer' (which we generated).
print(dataset_train[0])

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

{'prompt': 'In a survey of 200 students, it was found that x students like mathematics, y students like physics, and z students like chemistry. The number of students who like exactly two subjects is 45, and the number who like all three subjects is 12. If 25 students like both math and physics but not chemistry, 18 students like both physics and chemistry but not math, and 22 students like both math and chemistry but not physics, find the value of x + y + z given that exactly 30 students like none of the three subjects.', 'answer': '312'}


In [11]:
# Test your dataset format
print("Sample data:")
print(dataset_train[0])
print("\nKeys:", dataset_train[0].keys())
print("\nPrompt type:", type(dataset_train[0]['prompt']))
print("Answer type:", type(dataset_train[0]['answer']))

Sample data:
{'prompt': 'In a survey of 200 students, it was found that x students like mathematics, y students like physics, and z students like chemistry. The number of students who like exactly two subjects is 45, and the number who like all three subjects is 12. If 25 students like both math and physics but not chemistry, 18 students like both physics and chemistry but not math, and 22 students like both math and chemistry but not physics, find the value of x + y + z given that exactly 30 students like none of the three subjects.', 'answer': '312'}

Keys: dict_keys(['prompt', 'answer'])

Prompt type: <class 'str'>
Answer type: <class 'str'>


In [12]:
# Attach LoRA Adapters (PEFT)
print("🔗 Attaching LoRA adapters...")
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
    lora_dropout=0.05,
    bias="none",
)
# We wrap the model manually so GRPO knows it's a PEFT model
model = get_peft_model(model, peft_config)

🔗 Attaching LoRA adapters...


In [13]:
# --- MEMORY OPTIMIZATION MONKEY PATCH ---
# This fixes the OOM error by processing log-probs in mini-batches
import trl.trainer.grpo_trainer
import torch

def get_per_token_logps_chunked(model, input_ids, num_logits_to_keep, mini_batch_size=1):
    per_token_logps = []
    batch_size = input_ids.size(0)
    for i in range(0, batch_size, mini_batch_size):
        batch_end = min(i + mini_batch_size, batch_size)
        mini_input_ids = input_ids[i:batch_end]
        # The standard implementation calculates all logits at once, causing OOM
        # We do it in chunks:
        mini_logits = model(mini_input_ids, num_logits_to_keep=num_logits_to_keep + 1).logits
        mini_logits = mini_logits[:, :-1, :] # exclude last logit

        # Compute log probs
        log_probs = mini_logits.log_softmax(dim=-1)
        labels = mini_input_ids[:, -num_logits_to_keep:].unsqueeze(2)
        token_log_prob = torch.gather(log_probs, dim=2, index=labels).squeeze(2)
        per_token_logps.append(token_log_prob)
    return torch.cat(per_token_logps, dim=0)

# Apply the patch to the library function
# We set mini_batch_size=1 to be extremely safe with memory
trl.trainer.grpo_trainer.get_per_token_logps = lambda model, input_ids, num_logits_to_keep: get_per_token_logps_chunked(model, input_ids, num_logits_to_keep, mini_batch_size=1)
print("✅ Applied memory optimization patch to pooling layer!")


✅ Applied memory optimization patch to pooling layer!


In [14]:
# Load the TensorBoard extension
# %load_ext tensorboard
# %tensorboard --logdir gemma-2-2b-reasoning-v2 --port 6006 --bind_all
# Start TensorBoard pointing to your output directory
# (Make sure 'gemma-3-reasoning-output' matches the 'output_dir' in your GRPOConfig!)
# %tensorboard --logdir gemma-2-2b-reasoning-output

# Define Training Arguments (GRPO)
training_args = GRPOConfig(
    output_dir="gemma-2-2b-reasoning-v2",
    learning_rate=5e-6,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2, # Reverted to 2 (must be divisible by num_generations)
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    max_prompt_length=256,
    max_completion_length=200, # Reduced to save memory
    num_generations=2, # Reduced to 2 to prevent OOM
    
    # --- STEP MATH SETTINGS ---
    max_steps=330, 
    warmup_steps=30, # ~10% of total steps
    
    # --- VALIDATION SETTINGS ---
    eval_strategy="steps",      # Check validation set every X steps
    eval_steps=10,              # Check every 50 steps
    save_steps=10,              # Save a checkpoint every 50 steps
    logging_steps=2,
    
    fp16=False,
    bf16=True,
    report_to="tensorboard"
)

In [15]:
# 5. Initialize Trainer
# Note: We assume you still have your 'dataset' and reward functions from before
print("🚀 Starting GRPO Trainer...")
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[correctness_reward_func, hicra_planning_reward_func],
    args=training_args,
    train_dataset=dataset_train, # Your 630 questions
    eval_dataset=dataset_test,   # Your 50 test questions <--- ADDED THIS
)

🚀 Starting GRPO Trainer...


In [ ]:
# Train with GRPO
trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1}.


Step,Training Loss,Validation Loss


: 

In [ ]:
# Continue training from step 60 to step 180
trainer.args.max_steps = 180  # New target

# Resume from the last checkpoint
trainer_stats = trainer.train(resume_from_checkpoint=True)

In [ ]:
# Continue training from step 60 to step 180
trainer.args.max_steps = 270  # New target

# Resume from the last checkpoint
trainer_stats = trainer.train(resume_from_checkpoint=True)

Set up the transformers inference API:

1. Adjusting Your Script for the Project
Here is the adjusted script. I have updated it to fit the Gemma-9B context and added a safety step to clear memory before merging (crucial on cloud GPUs to avoid crashing at the finish line).

You should append this to the end of your training notebook/script.

2. Important Step for HF Spaces
You must add your Hugging Face Token as a Secret in the Space settings, or the script won't be able to push the model.

Go to your Space -> Settings.

Scroll to "Variables and secrets".

Add a New Secret: HF_TOKEN -> [Paste your Write token].

In [ ]:
import torch
import os
import gc
from huggingface_hub import login

# --- 1. MEMORY CLEANUP (Crucial for Cloud) ---
# RL Training fills VRAM. We need to clear it before the heavy "Merge" step.
print("🧹 Cleaning up VRAM before merging...")
try:
    del trainer
    del batch
    gc.collect()
    torch.cuda.empty_cache()
except:
    pass

# --- 2. RELOAD MODEL FOR MERGING ---
# Sometimes it's safer to reload the base model + adapter freshly to merge
# independent of the messy training state.
from unsloth import FastLanguageModel

print("🔄 Reloading model for clean merge...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "gemma-2-9b-reasoning-v1", # Your base model
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True,
)

# Load the adapters you just trained
# Assuming your GRPOConfig output_dir was "gemma-reasoning-output"
# and the latest checkpoint is saved there.
from peft import PeftModel
model = PeftModel.from_pretrained(model, "gemma-reasoning-output/checkpoint-final") # Update path to your actual checkpoint folder!

# --- 3. LOGIN & PUSH ---
hf_token = os.environ.get("HF_TOKEN")
if hf_token:
    login(token=hf_token)
else:
    print("⚠️ No HF_TOKEN found! Check your Space 'Settings' -> 'Variables' to add it.")

repo_name = "DataImaginations/Gemma-2-9B-Reasoning-v1" # Repo name might be `DataImaginations/` or `david-barnes`

print(f"⏳ Merging to 16-bit and Pushing to: {repo_name}...")

# This takes care of the de-quantization and merging in one go
model.push_to_hub_merged(
    repo_name,
    tokenizer,
    save_method = "merged_16bit", # options: "merged_4bit", "merged_16bit"
    token = hf_token
)

print("✅ Success! Your reasoning model is live.")

### 3. Configure LoRA:

Unsloth handles the target modules automatically (including the tricky gate_proj, up_proj, etc. that vanilla Peft requires you to list manually).

### Check where the model is stored

In [ ]:
# Check where the model is cached
from huggingface_hub import hf_hub_download
import os

cache_dir = os.path.expanduser("~/.cache/huggingface/hub/")
print(f"Model cache location: {cache_dir}")
print("\nContents:")
if os.path.exists(cache_dir):
    for item in os.listdir(cache_dir)[:10]:  # Show first 10 items
        print(f"  - {item}")
else:
    print("Cache directory not found yet")

# You can also set a custom cache location if you prefer:
# os.environ['HF_HOME'] = '/path/to/custom/cache'

## Apply QLora

Quick calculation:

700 records
Effective batch size = per_device_batch_size (2) × gradient_accumulation_steps (4) = 8
Steps per epoch = 700 / 8 = ~88 steps
So 60 steps = ~0.7 epochs - you haven't even completed one full pass through your data yet!

Recommendations:

Epochs |	Steps |	Use Case |
1 |	~90 |	Minimum - sees all data once |
2-3 |	~180-270|	Sweet spot for fine-tuning |
5+ |	440+ |	Risk of overfitting |

Since your loss was still decreasing at step 60, you probably have room to train more. I'd suggest trying max_steps = 180 (about 2 epochs) for a good balance.

Watch for:

✅ Good sign: Loss continues decreasing smoothly
⚠️ Overfitting warning: Loss drops very low (<0.1) or starts fluctuating

### LOGIN TO HUB

When we push to HuggingFace Hub, it will merge our local QLoRa adaptor with the base model we used to train, on the hub.

In [ ]:
import os
from huggingface_hub import login

# Try to login with token from environment variable
hf_token = os.getenv('HF_TOKEN')
if hf_token:
	login(token=hf_token)
	print("Logged in with HF_TOKEN environment variable") 
else:
	# Skip login for local training - you can still train without pushing to hub
	print("No HF_TOKEN found. Proceeding with local training on local GPU...")
	print("Note: You won't be able to push models to HuggingFace Hub without authentication")

# Push Model to hub!

In [ ]:
from unsloth import FastLanguageModel
import os
device = "cuda:0"

# 1. CONFIGURATION
# Point this to the exact folder on your disk
checkpoint_path = "outputs/checkpoint-180" 
repo_name = "DataImaginations/ministral-3B-Beancount-v1" # Your Hugging Face repo
hf_token = os.getenv('HF_TOKEN')

# 2. LOAD SPECIFIC CHECKPOINT
# Unsloth is smart: if you point it to a folder, it loads the base model 
# AND applies the adapters from that folder automatically.
print(f"📂 Loading checkpoint from {checkpoint_path}...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = checkpoint_path, 
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True, # Keep True for fast loading (Unsloth handles the merge magic)
)

# 3. MERGE & PUSH
# This will de-quantize the base model, merge your checkpoint-180 adapters, 
# and upload a clean 16-bit model to the Hub.
print(f"🚀 Merging and pushing to {repo_name}...")

model.push_to_hub_merged(
    repo_name,
    tokenizer,
    save_method = "merged_16bit", # options: "merged_4bit", "merged_16bit"
    token = hf_token
)

print("✅ Done! Your Junior Accountant (Checkpoint 180) is live!")